In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import tqdm.notebook as tqdm
import re
import traceback

In [2]:
def createURL(month, day):
    url = 'https://keirin.kdreams.jp/racecard/2021/' + str(month).zfill(2) + '/' + str(day).zfill(2) + '/'
    return url

In [54]:
seedURL = [ createURL(i, j) for i in range(4, 8, 1) for j in range(1, 30, 1)]

In [55]:
def get_race_id(sourceURLs):
    race_urls = {}
    for sourceURL in tqdm.tqdm(sourceURLs):
        try:
            req = requests.get(sourceURL)
            soup = BeautifulSoup(req.content, 'html.parser')
            time.sleep(1)
            race_html = soup.find_all('a', class_='JS_POST_THROW')
            for html in race_html:
                url = html.get('href')
                if 'racedetail' in url:
                    race_id = re.sub(r'\D', '', url)
                    race_urls[race_id] = url
        except:
            break
    return race_urls

In [56]:
race_urls = get_race_id(seedURL)

['https://keirin.kdreams.jp/racecard/2021/04/01/',
 'https://keirin.kdreams.jp/racecard/2021/04/02/',
 'https://keirin.kdreams.jp/racecard/2021/04/03/',
 'https://keirin.kdreams.jp/racecard/2021/04/04/',
 'https://keirin.kdreams.jp/racecard/2021/04/05/',
 'https://keirin.kdreams.jp/racecard/2021/04/06/',
 'https://keirin.kdreams.jp/racecard/2021/04/07/',
 'https://keirin.kdreams.jp/racecard/2021/04/08/',
 'https://keirin.kdreams.jp/racecard/2021/04/09/',
 'https://keirin.kdreams.jp/racecard/2021/04/10/',
 'https://keirin.kdreams.jp/racecard/2021/04/11/',
 'https://keirin.kdreams.jp/racecard/2021/04/12/',
 'https://keirin.kdreams.jp/racecard/2021/04/13/',
 'https://keirin.kdreams.jp/racecard/2021/04/14/',
 'https://keirin.kdreams.jp/racecard/2021/04/15/',
 'https://keirin.kdreams.jp/racecard/2021/04/16/',
 'https://keirin.kdreams.jp/racecard/2021/04/17/',
 'https://keirin.kdreams.jp/racecard/2021/04/18/',
 'https://keirin.kdreams.jp/racecard/2021/04/19/',
 'https://keirin.kdreams.jp/rac

In [57]:
print(len(race_urls))

8457


{'2720210401010001': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010001/',
 '2720210401010002': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010002/',
 '2720210401010003': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010003/',
 '2720210401010004': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010004/',
 '2720210401010005': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010005/',
 '2720210401010006': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010006/',
 '2720210401010007': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010007/',
 '2720210401010008': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010008/',
 '2720210401010009': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010009/',
 '2720210401010010': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010010/',
 '2720210401010011': 'https://keirin.kdreams.jp/keiokaku/racedetail/2720210401010011/',
 '2720210401010012': 'https://ke

In [58]:
main_colum = ['予想', '好気合', '総評', '枠番', '車番', '選手名府県/年齢/期別', '級班', '脚質', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外']
result_colum = ['予想', '着順', '車番', '選手名', '着差', '上り', '決まり手', 'S/B', '勝敗因']

In [59]:
def scrape_race_result(race_urls, pre_race_results={}):
    race_results = pre_race_results
    for race_id, url in tqdm.tqdm(race_urls.items()):
        if race_id in race_results.keys():
            continue       
        try:
            main = pd.read_html(url)
            df = main[4][:-1]
            df.columns = main_colum
            result_table = main[-2]
            result_table.columns = result_colum
            df_result = result_table.loc[ : , ['着順', '車番']]
            df = df.astype(str)
            df_result = df_result.astype(str)
            df = pd.merge(df_result, df, on='車番', how='left')
            race_results[race_id] = df
            time.sleep(1)
        except IndexError:
            print('IndexError: {}', url)
            continue
        except KeyError:
            print('keyerror: {}', url)
            continue
        except ValueError:
            print("ValueError: {}", url)
            continue
        except :
            traceback.print_exc()
            break
    return race_results

In [87]:
results = scrape_race_result(race_urls, results)

ValueError: {} https://keirin.kdreams.jp/hofu/racedetail/6320210410020002/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030001/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030002/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030003/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030004/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030005/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030006/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030007/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030008/
ValueError: {} https://keirin.kdreams.jp/ogaki/racedetail/4420210410030009/
ValueError: {} https://keirin.kdreams.jp/takeo/racedetail/8420210422020007/
ValueError: {} https://keirin.kdreams.jp/nara/racedetail/5320210518030007/
ValueError: {} https://keirin.kdreams.jp/nara/racedetail/5320210518030008/
ValueError: {} 

In [88]:
len(results)

8440

In [89]:
for key in results.keys():
    results[key].index = [key]*len(results[key])

In [90]:
race_results = pd.concat([results[key] for key in results.keys()], sort=False)

In [91]:
race_results

,着順,車番,予想,好気合,総評,枠番,選手名府県/年齢/期別,級班,脚質,ギヤ倍数,競走得点,1着,2着,3着,着外
2720210401010001,1,3,◎,nan,1,3,鰐淵 圭佑 群 馬/37/90,A3,追,3.92,78.33,9,8,6,3
2720210401010001,2,2,×,nan,3,2,岡崎 徹 神奈川/49/70,A3,追,3.86,72.77,4,5,4,14
2720210401010001,3,1,○,nan,3,1,中山 健 新 潟/37/89,A3,両,3.92,73.66,16,15,7,30
2720210401010001,4,5,注,nan,3,5,角口 聖也 千 葉/32/94,A3,逃,3.92,70.50,3,6,10,29
2720210401010001,5,4,▲,nan,5,4,志村 正洋 神奈川/46/77,A3,追,3.85,67.45,1,5,10,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8320210728020012,3,6,nan,nan,5,6,中村 昌弘 広 島/44/81,S2,追,3.92,98.90,3,12,12,36
8320210728020012,4,2,×,nan,2,2,村上 博幸 京 都/42/86,S1,追,3.92,110.62,14,3,5,33
8320210728020012,5,4,注,nan,4,4,松岡 篤哉 岐 阜/39/97,S2,逃,3.92,101.88,15,10,7,39
8320210728020012,6,3,▲,nan,4,3,吉村 和之 岐 阜/44/80,S2,追,3.93,100.15,3,8,8,33


In [92]:
race_results["枠番"].unique()

array(['3', '2', '1', '5', '4', '6'], dtype=object)

In [93]:
race_results.to_pickle("data/race_data.pkl")